# The Retrievability Consortium Parametric Analysis

On this notebook, the formalism as described on the "Retrievability Consoritum" 
cryptoecon section is implemented. Choices of priors were selected and propagated
so that the equilibrium values of the parameters according to the rationality
criteria are generated.

Based the equilibrium values distribution.

## Dependences

In [33]:
import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

## PyMC3 Model & Simulation

In [43]:
FIL = float

with pm.Model() as model:

    AVERAGE_PAYMENT: FIL = 1.0
    STD_PAYMENT: FIL = 1.0

    # Observables
    p: FIL = pm.Gamma('payment', mu=AVERAGE_PAYMENT, sd=STD_PAYMENT)
    ## Probability of Retrieving Given Non-Deal
    P_r_d2bar = pm.Uniform('P_r_d2bar', lower=0.5, upper=1)
    ## Probability of Retrieving Given Deal
    P_r_d2 = pm.Uniform('P_r_d2', lower=P_r_d2bar, upper=1)
    ## Probability of Retrieving Given Appeal
    P_r_d3 = pm.Uniform('P_r_d3', lower=0.0, upper=1.0)
    ## Probability of Slashing Given Deal
    P_d4_d2 = pm.Beta('P_d4_d2', mu=0.01, sd=0.01)

    # Inobservables
    ## Expected Number of Appeals on an Deal
    N_a = pm.Gamma('N_a', mu=P_d4_d2, sd=0.005)
    ## Deal Leverage on Utility
    alpha = pm.Uniform('alpha', lower=0.01, upper=2)

    # Parameters
    ## Committee Fee
    m_c = pm.Uniform('m_c', lower=0, upper=10)
    ## Slashing Multiplier
    m_s = pm.Uniform('m_s', lower=0, upper=4_000)

    # Auxiliary Variables
    ## Retrieval Utility
    pi_r_C: FIL = pm.Deterministic('pi_r_C', p / alpha)
    ## Probability of Slashing Given an Appeal
    P_d4_d3 = pm.Uniform('P_d4_d3', lower=P_r_d3, upper=1)
    ## Probability of Retrieving Given that there's no Appeal
    P_r_d3bar = pm.Uniform('P_r_d3bar', lower=0.0, upper=P_r_d3)
    ## Probability of Not Slashing Given an Deal
    P_d4bar_d2 = pm.Deterministic('P_d4bar_d2', 1 - P_d4_d2)
    ## Marginal Probability on Deal
    d_r = P_r_d2 - P_r_d2bar
    ## Equilibrium Committee Fee
    eq_mc = pm.Deterministic('eq_mc', (d_r + alpha + 2 * P_d4_d2 - 1) / N_a)
    ## Equilibrium Slashing Multiplier
    eq_ms = pm.Deterministic('eq_ms', 1 / P_d4_d2 - 1)

    # Payoffs & Decisions
    pi_d1_C = pi_r_C * P_r_d2 + p * (2 * P_d4_d2 - N_a * m_c - 1)
    pi_d1_C_bar = pi_r_C * P_r_d2bar
    U_d1 = pm.Deterministic('U_d1', pi_d1_C - pi_d1_C_bar)
    d1 = pm.Deterministic('d1', U_d1 > 0)

    pi_d2_P = p * (P_d4bar_d2 - m_s * P_d4_d2)
    pi_d2_P_bar = 0.0
    U_d2 = pi_d2_P - pi_d2_P_bar
    d2 = pm.Deterministic('d2', d1 & (U_d2 > 0))

    pi_d3_C = pi_r_C * P_r_d3 + p * (P_d4_d3 - m_c)
    pi_d3_C_bar = pi_r_C * P_r_d3bar
    U_d3 = pi_d3_C - pi_d3_C_bar
    d3 = pm.Deterministic('d3', d2 & (U_d3 > 0))

    pi_d4_R = pm.Bernoulli('pi_d4_R', p=1-P_r_d3)
    pi_d4_R_bar = 0 if pi_d4_R == 1 else 0
    U_d4 = pi_d4_R - pi_d4_R_bar
    d4 = pm.Deterministic('d4', d3 & (U_d4 > 0))

    # Priors for unknown model parameters
    trace = pm.sample(draws=20_000,
                      chains=4,
                      tune=1_000,
                      nuts={'target_accept':0.99},
                      return_inferencedata=True)


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [P_r_d3bar, P_d4_d3, m_s, m_c, alpha, N_a, P_d4_d2, P_r_d3, P_r_d2, P_r_d2bar, payment]
>BinaryGibbsMetropolis: [pi_d4_R]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 590 seconds.
There were 223 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9660238247394375, but should be close to 0.99. Try to increase the number of tuning steps.
There were 29 divergences after tuning. Increase `target_accept` or reparameterize.
There were 209 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9715171673043769, but should be close to 0.99. Try to increase the number of tuning steps.
There were 72 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


## Analysis

In [36]:
df = trace.posterior.to_dataframe()

metric_0 = df.d1
metric_1 = df.d2[metric_0 == True]
metric_2 = (df.P_r_d2 / df.P_r_d2bar)[metric_0 == True]
#metric_3 = df.payment * (1 - df.P_r_d2) * df.P_d4_d2 / df.pi_r_C
#metric_4 = df.pi_d4_R_bar
metric_5 = df.pi_d4_R

### Studying the effect of $m_s$ on the decisions

In [47]:
var = 'm_s'

fig_df = df
fig = px.histogram(fig_df, x=var, facet_col='d1', histnorm='probability density')
fig.show()

fig_df = df.query('d1 == True')
fig = px.histogram(fig_df, x=var, facet_col='d2', histnorm='probability density')
fig.show()

fig_df = df.query('d1 == True & d2 == True')
fig = px.histogram(fig_df, x=var, facet_col='d3', histnorm='probability density')
fig.show()

### Studying the effect of $m_c$ on the decisions

In [42]:
var = 'm_c'

fig_df = df
fig = px.histogram(fig_df, x=var, facet_col='d1', histnorm='probability density')
fig.show()

fig_df = df.query('d1 == True')
fig = px.histogram(fig_df, x=var, facet_col='d2', histnorm='probability density')
fig.show()

fig_df = df.query('d1 == True & d2 == True')
fig = px.histogram(fig_df, x=var, facet_col='d3', histnorm='probability density')
fig.show()